# Image Analysis 2021

This Jupyter notebook is part of the course Image Analysis from Radboud University (Nijmegen, Netherlands), and it was developed by researchers of Radboud University Medical Center (Nijmegen, Netherlands).

You should have obtained this notebook by downloading it from the official Brightspace page of the course.
If this is not the case, you should contact the course coordinator at this email address: geert.litjens@radboudumc.nl

This notebook formulates an assignment as part of the course, and the content of this notebook should be used solely to develop a solution to this assignment.
You should not make the code provided in this notebook, or your own solution, publicly available.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` by substituting `None` variables or by adding your own solution and any place that says "YOUR ANSWER HERE" with your answers to the questions. Note that it is perfectly fine to substitute the images in the exercises with your own if you want to. Please fill in your name and collaborators below:

## Students
Please fill in this cell with your name and e-mail address. This information will be used to track completion of the assignments.

* Name student #1, email address: ...
* Name student #2, email address (optional): ...

## Instructions

* Groups: You should work in **groups of maximum 2 people**.
* Deadline for this assignment: 
 * Preferably before May 12th
 * Please upload your **fully executed** notebook to BrightSpace
  The file name of the notebook you submit must be ```NameSurname1_NameSurname2.ipynb```

This notebooks contains cells with snippets of code that we provide in order to load and visualize data, but also some convenience functions that could be useful to develop your assignment.

We also provide templates for functions that have to be implemented, with a given list of input variables and some output variables.

Your submission should contain the **fully executed** notebook with **your code** implemented, as well as **your answers** to questions.

## Libraries

First, we import the basic libraries necessary to develop this assignment.

In [ ]:
import skimage as ski # For reading images
import skimage.segmentation # For the segmentation exercise
from skimage.segmentation._slic import _enforce_label_connectivity_cython # Efficient label connectivity checking

from sklearn import datasets as skldatasets # Example datasets for machine learning
from sklearn import neighbors # KNN classifier
from sklearn.calibration import CalibratedClassifierCV # Needed to get probabilities from SVM classifiers
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import pairwise_distances_argmin # For the color quantization section
from sklearn.model_selection import train_test_split #
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from scipy.special import expit
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils import shuffle

import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt # Plotting and visalization of images
import numpy as np # Basic math and array functions

# K-means clustering

Although K-means clustering typically is used to identify clusters in an unsupervised fashion, it also has some surprising use-cases. First, we will revisit our quantization exercise from Week 1. To make it a bit more challenging, we will look at color image quantization this time.

## Color Quantization

Let's again start by loading a nice image:

In [ ]:
# Load the Summer Palace photo
china = skldatasets.load_sample_image("china.jpg")
fig, axes = plt.subplots(1, 1, figsize=(12, 8)) 
axes.imshow(china); # Show the orignal image

Below is a very simple quantization function which simply limits every channel (R, G, and B) to a predetermined number of values linearly.

In [ ]:
def quantize_image(image, nr_of_values):
    values_per_channel = np.ceil(np.cbrt(nr_of_values)).astype("ubyte")
    quantized_image = np.array([np.round(values_per_channel * (image[:, :, i].astype("float") / 255.)) * (255 // values_per_channel) for i in range(3)]).transpose(1,2,0)
    return quantized_image.astype("ubyte") # Make sure the image is stored as integer for visualization

A standard RGB image can have 255*255*255 different colors, so more than 16 million. Let's try to reduce that to 27, so 3 values per color channel.

In [ ]:
nr_values = 27
quant_china = quantize_image(china, nr_values)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(24, 8))
axes[0].imshow(china)
axes[1].imshow(quant_china);

<font color="blue">**Assignment:** Now let's try to select these 27 colors in a way that is a bit smarter. The idea is that we can use the K-means algorithm to find the largest clusters of color values and use those cluster centers as the best values for quantization. The below cell should perform this task. Modify it by using the `KMeans` class so that the function completes. Remember that you can get the manual by using `?KMeans`.

In [ ]:
# We first need to reshape the image to a list of pixels instead of a 2D image
w, h, d = original_shape = china.shape
image_array = np.reshape(china, (w * h, d))

# We perform the clustering on a subset of pixels to make it complete a bit faster. You can also identify the clusters using all the pixels to be a bit more accurate
image_array_sample = shuffle(image_array, random_state=0)[:10000]

# Your code here, replace the None's below
kmeans = None
kmeans.fit(image_array_sample)
labels = kmeans.predict(image_array)

Given the predicted labels, the function below recreates the image with just the 27 color values.

In [ ]:
def recreate_image(cluster_centers, labels, w, h):
    """Recreate the (compressed) image from the cluster centers & labels"""
    image = np.zeros((w, h, 3))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = cluster_centers[labels[label_idx]]
            label_idx += 1
    return image

Now we recreate te actual image:

In [ ]:
quant_china_kmeans = recreate_image(kmeans.cluster_centers_, labels, w, h)

Plot the figures to allow for a visual comparison

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(36, 16)) 
axes[0][0].imshow(china)
axes[0][1].imshow(quant_china_kmeans.astype("ubyte"))
axes[0][2].imshow(quant_china)
axes[1][0].imshow(china[200:300, 150:250])
axes[1][1].imshow(quant_china_kmeans[200:300, 150:250].astype("ubyte"));
axes[1][2].imshow(quant_china[200:300, 150:250]);

<font color="blue">**Question:** Experiment a bit with different number of values. How many values do you need to make the image visually (almost) the same as the original? Could you come up with a way to quantify the difference between the original and the quantified versions?

YOUR ANSWER HERE

## Superpixel Segmentation

Another atypical application area of KMeans is to perform so-called superpixel segmentation of an image. This identifies visually similar, spacially close regions and labels them. In downstream tasks, features could be extracted from these regions to perform classification and thus segmentation. To give us a bit more freedom on how to conduct this segmentation, you can find a simple, relatively inefficient implementation of KMeans. 

In [ ]:
def KMeans(features, centroids, nr_iterations=10, metric="euclidean"):
    for i in range(nr_iterations): # Perform a number of iterations, each time updating the cluster centers
        clusters = []
        for feature_vector in features: # Calculate the distance between every pixel and every cluster center
            dist = np.sqrt(np.sum(np.square(centroids - feature_vector), axis=-1))
            clusters.append(np.argmin(dist)) # Store the cluster label for each pixel
        for c in range(len(centroids)):
            centroids[c] = np.mean(features[np.where(np.array(clusters) == c)], axis=0)
        # Check whether any cluster assignments changed since last iteration. If not, stop!
        if i == 0:
            old_clusters = np.array(clusters).copy()
        else:
            if (old_clusters == np.array(clusters)).all():
                break
            else:
                old_clusters = np.array(clusters).copy()
    return clusters, centroids

The SLIC algorithm is implemented below. It converts an image to a list of feature vector, one entry for each pixel. The features consist of spatial positions (x, y) and color values (r, g, b), so five in total. Subsequently, the cluster centroids for K-means are initialized on a grid across the image. By subsequently applying K-means to the feature list, every pixels will be assigned to a cluster center, which thanks to the positional features will be spatially close, and similar in color.

In [ ]:
def SLIC(image, nr_superpixels=100, nr_iterations=10, metric="euclidean"):
    image_float = image / 255. # To make the positions and the colors have roughly the same value range (0 - 1), divide by 255
     # Generate the position features
    row_pos = np.arange(0, 1., 1./image.shape[0])
    col_pos = np.arange(0, 1., 1./image.shape[1])
    xx, yy = np.meshgrid(col_pos, row_pos)
    indices = np.array([yy,xx]).transpose(1,2,0)
    
    # Combine the positions features and the color features
    feat_matrix = np.concatenate([indices, image_float], axis=2).reshape(-1, 5)
    
    # Now initialize the centroids. First determine how many clusters there should be per row and column
    superpixels_per_row = np.ceil(np.sqrt(nr_superpixels) * (image.shape[1] / image.shape[0]))
    superpixels_per_col = np.ceil(nr_superpixels / superpixels_per_row)
    step_size_x = image.shape[1] / superpixels_per_row
    step_size_y = image.shape[0] / superpixels_per_col
    centroid_row_pos = np.arange(step_size_x / 2, image.shape[1], step_size_x).astype("int")
    centroid_col_pos = np.arange(step_size_y / 2, image.shape[0], step_size_y).astype("int")
    centroids_xx, centroids_yy = np.meshgrid(centroid_row_pos, centroid_col_pos)
    centroid_rgb_vals = image_float[centroids_yy, centroids_xx]
    centroid_indices = np.array([centroids_yy/image_float.shape[0],centroids_xx/image_float.shape[1]]).transpose(1,2,0)
    centroid_feat_matrix = np.concatenate([centroid_indices, centroid_rgb_vals], axis=2).reshape(-1,5)
    
    # Perform the K-means given the pixel feature matrix and the centroids
    clusters, centroid_feat_matrix = KMeans(feat_matrix, centroid_feat_matrix, nr_iterations, metric)
    
    # Removes the small cluster and ensure connectivity 
    segment_size = (image.shape[0] * image.shape[1]) // nr_superpixels
    result = _enforce_label_connectivity_cython(np.array(clusters).reshape(image.shape[0], -1).astype(np.intp)[None], int(0.5 * segment_size), 3 * segment_size)
    return result[0]

Perform the SLIC algorithm on a subimage to reduce the waiting time for the algorithm to complete:

In [ ]:
china_sub = china[0:200,0:200]
china_slic = SLIC(china_sub)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8)) 
axes[0].imshow(china_sub)
axes[1].imshow(skimage.segmentation.mark_boundaries(china_sub, china_slic));

<font color="blue">**Assignment:** There are two things in the K-means function for SLIC specifically that are not implemented yet. The first is that the spatial position features and color features cannot be weighted, i.e. they all contribute equally to the distance. However, there are 3 color features and 2 spatial features, thus color features play to big a role. Adept the function to allow for separate weighting of the spatial and color features and investigate how this changes the results. What happens to the superpixels when you make the weighting for the position features higher? The second limitation is that currently only the Euclidean distance is implemented. This limits our flexibility. Implement at least one other distance metric (e.g. City Block Distance). You can use the metric parameter to allow for selection. 

YOUR ANSWER HERE

## kNN classification

The last assignment of this section will focus on supervised nearest neighbors algorithms, specifically k-Nearest Neighbor Classification. kNN determines a label for new points by determining which neighbors are closed. The amount of neighbors has to be specified, the afformentioned `k`. Let's inspect the effect of the value of `k` on the decision boundary of the kNN classifier.

The function below we will use throughout the rest of the exercises to plot the decision boundaries in conjuction with the data for each classifier.

In [ ]:
def plot_decision_boundary(X, y, clfs, X_test=None, y_test=None):
    if not isinstance(clfs, list):
        clfs = [clfs]
    # preprocess dataset, split into training and test part
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.02
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    if max(y) == 1:
        cm = plt.cm.RdYlBu        
        cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    else:
        cm = plt.cm.RdYlBu
        cm_bright = ListedColormap(['#FF0000', '#FFFF00', '#0000FF'])
    plt.figure(figsize=(8 * len(clfs), 8))
    ax = plt.subplot(1, len(clfs)+1, 1)
    ax.set_title("Input data")
    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright,
               edgecolors='k')
    if X_test is not None and y_test is not None:
        # Plot the testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
                   edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())

    for i, clf in enumerate(clfs):        
        ax = plt.subplot(1, len(clfs)+1, i+2)
        ax.set_title(clf.__str__().split("(")[0])        
        if X_test is not None and y_test is not None:
            score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        if y.max() > 1:
            Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

            # Put the result ianto a color plot
            Z = Z.reshape(xx.shape)
            ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)
        else:
            if hasattr(clf, "predict_proba"):
                Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
            elif hasattr(clf, "decision_function"):
                Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
            else:
                Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

            # Put the result into a color plot
            Z = Z.reshape(xx.shape)
            ax.contourf(xx, yy, Z, cmap=cm, alpha=.8, levels=[-1.0,  -0.25, 0, 0.25, 0.5, 0.75, 1.0, 1.25, 2.0])
            CS = ax.contour(xx, yy, Z, colors = "black", levels=[-1.0, -0.25, 0, 0.25, 0.5, 0.75, 1.0, 1.25, 2.0])
            ax.clabel(CS, CS.levels, inline=True, fontsize=10)
        # Plot the training points
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright,
                   edgecolors='k')
        # Plot the testing points
        if X_test is not None and y_test is not None:
            ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                       edgecolors='k', alpha=0.6)    
            

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if X_test is not None and y_test is not None:
            ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                    size=15, horizontalalignment='right')
    plt.show()

The cell below loads some example data and then trains two kNN classifier, one with `k=1` and one with `k=40`.

In [ ]:
# import some data to play with
iris = skldatasets.load_iris()

# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target

# we create an instance of Neighbours Classifier and fit the data.
knn_1 = neighbors.KNeighborsClassifier(1)
knn_1.fit(X, y)

knn_40 = neighbors.KNeighborsClassifier(40)
knn_40.fit(X, y)

plot_decision_boundary(X, y, [knn_1, knn_40])

<font color="blue">**Question:** Describe the difference you see in the decision boundaries. What would you expect the effect to be on generalization? How could you test that? If you have a good idea, you could try to adapt the cell above and test which `k` is better for this dataset.

YOUR ANSWER HERE

# Linear classifiers

Now let's change gears and look at parametric classifiers. We will first investigate the differences between three different linear classifier: standard linear regression, logistic regression and linear support vector classifiers (SVC). First, let's implement a simple one variable linear regression, which can be solved analytically.

<font color="blue">**Assignment:** Fix the class below by replacing the `None` statements in the `fit` function. You can either have a look at the lecture slides or inspect this page on Wikipedia: https://www.wikiwand.com/en/Simple_linear_regression

In [ ]:
class SimpleLinearRegression:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
    
    def fit(self, X, y):
        # YOUR CODE HERE # 
        self.coef_ = None
        self.intercept_ = None
    
    def predict(self, X):
        return self.coef_ * X + self.intercept_,

Once you have managed to implement the linear regression, let's do a comparison in a one variable case to logistic regression. The dell below generates a simple random dataset with two classes to experiment with.

In [ ]:
X, y = skldatasets.make_blobs(n_samples=40, n_features=1, centers=2, random_state=0)

In the cell below, we will fit our model and a logistic regression model to the data. We will then plot the fitted models for X and y.

In [ ]:
ols = SimpleLinearRegression()
ols.fit(X, y)
plt.figure(figsize=(8, 6))
plt.plot(sorted(X[:, 0]), ols.coef_ * sorted(X[:, 0]) + ols.intercept_, linewidth=1)
plt.axhline(.5, color='.5')

# Fit the classifier
clf = linear_model.LogisticRegression()
clf.fit(X, y)

# and plot the result
plt.scatter(X.ravel(), y, color='black', zorder=20)

loss = expit(sorted(X[:, 0]) * clf.coef_ + clf.intercept_).ravel()
plt.plot(sorted(X[:, 0]), loss, color='red', linewidth=3)

plt.ylabel('y')
plt.xlabel('X')
plt.xticks(np.arange(X.min(), X.max()))
plt.yticks([0, 0.5, 1])
plt.ylim(-.25, 1.25)
plt.legend(('Logistic Regression Model', 'Linear Regression Model'),
           loc="lower right", fontsize='small')
plt.tight_layout()
plt.show()

<font color="blue">**Question:** Based on the loss function plotted above, describe the disadvantage of using a regular regression model for classification. What are the advantages of logistic regression?

YOUR ANSWER HERE

Now we will make it slightly more complex and look at two features and also include linear SVM. 

In [ ]:
X, y = skldatasets.make_blobs(n_samples=40, n_features=2, centers=2, random_state=0)

In [ ]:
clf_logr = linear_model.LogisticRegression().fit(X,y)
clf_linr = linear_model.LinearRegression().fit(X, y)
clf_linsvm = CalibratedClassifierCV(LinearSVC(C=100, loss="hinge", random_state=42, max_iter=10000)).fit(X, y)
plot_decision_boundary(X,y,[clf_linr, clf_logr, clf_linsvm])

<font color="blue">**Question:** Describe the differences you see in the classifier boundaries. What happens when you change the C parameter of the linear SVM? Can you predict what its use is? Which classifier would you typically use as first choice and why?

YOUR ANSWER HERE

# Decision trees and random forests

The last exercise of this week will focus around decision trees and ensembles of decision trees (also called Random Forests). Decision trees have some very attractive advantages: they are simple to implement, they are explainable and can be quite accurate. However, they are also notorious for overfitting (i.e. getting perfect results on the training data but performing poorly on new data). Let's explore how they behave in practice.

The cell below loads a toy dataset and first an unconstrained decision tree. We subsequently plot the decision boundary and a graph representation.

In [ ]:
# Load data
iris = skldatasets.load_iris()

# Pick the first two features
X, y = shuffle(iris.data[:, :2], iris.target, random_state=0)
X_train = X[:100,:2]
X_test = X[100:,:2]
y_train = y[:100]
y_test = y[100:]

# Train
clf = DecisionTreeClassifier().fit(X_train, y_train)
plot_decision_boundary(X, y, clf)

plt.figure(figsize=(24,24))
plot_tree(clf, filled=True, feature_names=iris.feature_names, fontsize=10)
plt.show()

As you can see an unconstrained decision tree become quite complex, let's have a look at how it performs on unseen data:

In [ ]:
print("Accuracy on the training data: " + str(np.sum(clf.predict(X_train) == y_train)/len(y_train)))
print("Accuracy on the test data: " + str(np.sum(clf.predict(X_test) == y_test)/len(y_test)))

<font color="blue">**Question:** As you can see, there is quite a big gap in accuracy. Let's try to constrain the tree and make this gap smaller. You have to modify `DecisionTreeClassifier()` by providing additional parameters. An example of the one you could test is max_depth, but there are also others. Try to get the best possible test accuracy and visualize the tree and decision boundary. What happens? Can you explain what the parameters you tried did? What was the highest test accuracy you could achieve?

YOUR ANSWER HERE

The last exercise will compare all the classifier we have encountered up till now in addition to the Random Forest, an ensemble of decision trees. We will inspect their performance on diverse datasets.

In [ ]:
# This is the set of classifiers we will explore
classifiers = [
    neighbors.KNeighborsClassifier(n_neighbors=3),
    CalibratedClassifierCV(LinearSVC()),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)]

X, y = skldatasets.make_classification(n_samples=500, n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [skldatasets.make_moons(n_samples=500, noise=0.3, random_state=0),
            skldatasets.make_circles(n_samples=500, noise=0.2, factor=0.5, random_state=1),
            linearly_separable
            ]
# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)

    trained_classifiers = []
    for clf in classifiers:
        clf.fit(X_train, y_train)
        trained_classifiers.append(clf)
        
    plot_decision_boundary(X_train, y_train, trained_classifiers, X_test, y_test)

<font color="blue">**Question:** By looking at the different datasets and the results of the classifiers (accuracy in the bottom right of each graph), you should be able to identify some weaknesses for specific classifiers. What classifier performance the best overal? Where does each classifier excel? You can experiment a bit with the dataset functions to see if you can discover anything else.

YOUR ANSWER HERE